In [ ]:
import os #This package allows you to interact with your operating system
import numpy as np #standard math library: https://numpy.org/doc/stable/
import matplotlib.pyplot as plt #standard plotting library: https://matplotlib.org/stable/index.html
from astropy.io import fits #Astropy is a multi-purpose python package made by astronomers: https://docs.astropy.org/en/stable/index.html
from astropy.stats import * #Astropy is massive and we only want specific tools from it for now, so it's best to import only what we need.
from photutils.background import Background2D, MedianBackground #Photutils is another package used to manipulate images and find sources_B in our images.
from photutils.detection import DAOStarFinder #Photutils documentation: https://photutils.readthedocs.io/en/stable/
from photutils.detection import IRAFStarFinder
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry
import pandas as pan
import astroalign as aa
import warnings
warnings.filterwarnings('ignore')

def magnitude_to_flux (m, filter = "none"): #Vega magnitude
    reference_flux = [756.1, 1392.6, 995.5, 702.0, 452.0] #data from Bessell et al. (1998)
    reference_mag = [0.030, 0.035, 0.035, 0.075, 0.095]# (Johnson, 1964, 1965) average
    if filter == "none" :
        filter = str( input ("Which filter ? (U,B,V,R,I) :"))
    list_filters = ["U","B","V","R","I"]
    filter_index = list_filters.index (list(set(list_filters).intersection(set(filter)))[0])
    m_vega = reference_mag [filter_index]
    flux_vega = reference_flux [filter_index]
    flux = np.power (10, 0.4*(m_vega - m))*flux_vega
    return (flux)

def get_data_header (image_path): 
    image = fits.open (image_path)
    primary_hdu = image['PRIMARY'].data
    header = image['PRIMARY'].header
    image.close ()
    return (primary_hdu, header)

def get_hdu_stats (image_data): #
    min = np.min (image_data)
    max = np.max (image_data)
    mean = np.mean (image_data)
    std = np.std (image_data)
    return (min, max , mean, std)

def plot_histogram (image_data):
    image_data_flat = image_data.flatten ()
    histogram = plt.hist (image_data_flat, bins = np.linspace(0,500,100))
    plt.show ()

def get_background_median (image_data):
    sigma_clip = SigmaClip(sigma=3.0, maxiters= 5) 
    bkg_estimator = MedianBackground()
    background = Background2D (image_data, box_size= (50,50), filter_size = (3,3),
    sigma_clip= sigma_clip, bkg_estimator= bkg_estimator)
    return (background.background_median)

def find_sources_DAO (image_data, FWHM, thresh, sharphi, mask, nb):
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    daofind = DAOStarFinder (fwhm= FWHM, threshold= thresh*std, 
    sharphi= sharphi, brightest= nb)
    sources_B = daofind (image_data - median, mask = mask)
    return (sources_B)

def find_sources_DAO_invert (image_data, FWHM, thresh, sharphi, mask):
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    daofind = DAOStarFinder (fwhm= FWHM, threshold= thresh*std, 
    sharphi= sharphi)
    sources_B = daofind (image_data - median, mask = mask)
    try :
        shape = image_data.shape
        for i in sources_B:
            i['xcentroid'] = shape[1] - i['xcentroid']
            i['ycentroid'] = shape[0] - i['ycentroid']
    except :
        print ("errorrrrr")
    return (sources_B)


def find_sources_IRAF (image_data, FWHM, thresh, sharphi):
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    irafind = IRAFStarFinder (fwhm= FWHM, threshold= thresh*std, 
    sharphi= sharphi)
    sources_B = irafind (image_data - median) 
    return (sources_B)

def show_image (image_data, vmin, vmax):
    plt.imshow(np.log(image_data), cmap='gray', vmin = vmin, 
    vmax = vmax)
    plt.colorbar ()
    plt.show ()

def plot_sources (image_data, sources_B, vmin, vmax):
    positions = np.transpose((sources_B['xcentroid'], sources_B['ycentroid']))
    apertures = CircularAperture(positions, r=10)
    plt.figure(figsize=(10,10))
    
    plt.imshow(np.log(image_data), origin='lower', cmap = "gray", 
    vmin=vmin, vmax = vmax)
    v = apertures.plot(color='red', lw=5, alpha=0.5)
    plt.show()

def find_deltas (image_data_B, image_data_V):
    mask_calibration = np.ones (image_data_V.shape, dtype = bool)

    mask_default = np.zeros (image_data_B.shape, dtype = bool) 
    mask_calibration[1810:1850, 370:410] = False
    mask_calibration[925:965, 615:655] = False 
    mask_calibration[665:705, 2770:2810] = False 
    mask_calibration[1560:1600, 2600:2640] = False 

    sources_B = find_sources_DAO (image_data_B, 5, 3.5, 2, mask = mask_calibration, nb = 4)
    sources_V = find_sources_DAO (image_data_V, 5, 3.5, 2, mask = mask_calibration, nb = 4)

    reference_list_B = [[x['xcentroid'] for x in sources_B],[y['ycentroid'] for y in sources_B]]
    reference_list_V = [[x['xcentroid'] for x in sources_V],[y['ycentroid'] for y in sources_V]]

    delta_x_list = [reference_list_B[0][i] - reference_list_V[0][i] for i in range(len(reference_list_B[0]))]
    delta_y_list = [reference_list_B[1][i] - reference_list_V[1][i] for i in range(len(reference_list_B[1]))]
    delta_x = np.mean (delta_x_list)
    delta_y = np.mean (delta_y_list)


    return (delta_x, delta_y) 

def find_close_sources (sourceA, sourceB, threshold):
    n = 0
    sources_list1 = []
    sources_list2 = []
    V_mag_list = []
    mag_list = []

    threshold = float (threshold)
    for i in sourceA :
        for j in sourceB :
            if ((np.abs(i['xcentroid'] - j['xcentroid']) <=  threshold) and (np.abs (i['ycentroid'] - j['ycentroid']) <= threshold)) :
                sources_list1.append (i)
                sources_list1.append (j)

                n += 1
                mag_list.append(i['mag'] - j['mag'])
                V_mag_list.append (j['mag'])

    return (n, sources_list1, sources_list2, mag_list, V_mag_list)

image_data_B, header_B = get_data_header ('~/M55_ImageB.fits')
image_data_V, header_V = get_data_header ('~/M55_ImageV.fits')

delta_x, delta_y = find_deltas (image_data_B, image_data_V)

mask_default = np.zeros (image_data_B.shape, dtype = bool)

sources_B = find_sources_DAO (image_data_B, 5, 3.5, 2, mask = mask_default, nb = 5000)
sources_V = find_sources_DAO (image_data_V, 5, 3.5, 2, mask = mask_default, nb = 5000)

for i in sources_V:
    i['xcentroid'] += delta_x
    i['ycentroid'] += delta_y

print (sources_B, sources_V)
print ("B", sources_B)
print ("V", sources_V)

positions = np.transpose((sources_B['xcentroid'], sources_B['ycentroid']))
apertures = CircularAperture(positions, r=10)

positions_V = np.transpose((sources_V['xcentroid'], sources_V['ycentroid']))
apertures_V = CircularAperture(positions_V, r=10)

plt.subplot (2,1,1)
plt.imshow(np.log(image_data_B), origin='lower', cmap = "gray", vmin = 4.6, vmax = 5)
v = apertures.plot(color='red', lw=5, alpha=0.5)

plt.subplot(2,1,2)
plt.imshow(np.log(image_data_V), origin='lower', cmap = "gray", vmin = 5.5, vmax = 5.8)
v_V = apertures_V.plot(color='red', lw=5, alpha=0.5)

plt.show()

n, l1, l2, B_V_list, V_mag_list = find_close_sources(sources_B, sources_V, 0.5)
print(n)

plt.title("M55 H-R Diagram (Uncalibrated)")
plt.xlabel("Color Index (B-V)")
plt.ylabel(r"M$_{V}$")
plt.scatter (B_V_list, V_mag_list, s = 3)
plt.gca().invert_yaxis()

plt.show()




